## LenetでMNIST

#### 原論文
http://yann.lecun.com/exdb/publis/pdf/lecun-99.pdf
#### ChainerのMNISTチュートリアル
https://docs.chainer.org/en/stable/examples/mnist.html

In [7]:
import numpy as np
import chainer
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

In [8]:
class LeNet(Chain):
    def __init__(self):
        super(LeNet, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(
                in_channels=1, out_channels=6, ksize=5, stride=1)
            self.conv2 = L.Convolution2D(
                in_channels=6, out_channels=16, ksize=5, stride=1)
            self.fc3 = L.Linear(None, 84)
            self.fc4 = L.Linear(84, 10)

    def __call__(self, x):
        h = F.relu(self.conv1(x))
        h = F.max_pooling_2d(h, 2, 2)
        h = F.relu(self.conv2(h))
        h = F.max_pooling_2d(h, 2, 2)
        h = F.relu(self.fc3(h))
        if chainer.config.train:
            return self.fc4(h)
        return F.softmax(self.fc4(h))

In [9]:
train, test = chainer.datasets.mnist.get_mnist(ndim=3)
batchsize = 128

train_iter = iterators.SerialIterator(train, batchsize)
test_iter = iterators.SerialIterator(test, batchsize, False, False)

In [10]:
model = LeNet()
gpu_id = -1
max_epoch = 10
model = L.Classifier(model)
optimizer = optimizers.MomentumSGD()
optimizer.setup(model)
updater = training.updaters.StandardUpdater(train_iter, optimizer, device=gpu_id)
trainer = training.Trainer(updater, (max_epoch, 'epoch'), out='mnist_result')

In [11]:
from chainer.training import extensions
trainer.extend(extensions.LogReport())
trainer.extend(extensions.snapshot(filename='snapshot_epoch-{.updater.epoch}'))
trainer.extend(extensions.snapshot_object(model.predictor, filename='model_epoch-{.updater.epoch}'))
trainer.extend(extensions.Evaluator(test_iter, model, device=gpu_id))
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'validation/main/loss', 'validation/main/accuracy', 'elapsed_time']))
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
trainer.extend(extensions.dump_graph('main/loss'))

In [12]:
trainer.run()

epoch       main/loss   main/accuracy  validation/main/loss  validation/main/accuracy  elapsed_time
1           0.40473     0.875983       1.51474               0.964399                  51.2284       
2           0.103885    0.968817       1.49234               0.979233                  113.398       
3           0.0739544   0.977079       1.49047               0.980716                  174.218       
4           0.0601382   0.981153       1.48494               0.983881                  227.164       
5           0.0520363   0.983775       1.48491               0.983584                  278.809       
6           0.0460341   0.985674       1.48123               0.986254                  335.385       
7           0.0411847   0.987007       1.48095               0.985858                  391.664       
8           0.0359454   0.988598       1.47805               0.987935                  443.785       
9           0.0336231   0.989356       1.47759               0.98843                